In [ ]:
%cd ../

In [ ]:
import time

from plasma.parallel_processing import communicators, queues
from plasma.parallel_processing.communicators import TreeFlow
from plasma.functional import partials, AutoPipe

In [ ]:
def compute(x):
    time.sleep(2)
    return str(x)

def compute2(x):
    time.sleep(2)
    return x + 'c'

class ComputeClass(AutoPipe):

    def __init__(self, hihi, haha):
        super().__init__()
    
    def run(self, tmp):
        return tmp + 'd'

In [ ]:
out_queue= queues.TransferQueue()

class Flow(TreeFlow):

    def __init__(self):
        super().__init__()

        self.distributor = communicators.distributors.IteratorDistributor()
        self.compute = compute
        self.compute2 = compute2
        self.compute3 = ComputeClass(1, 2)
        self.printer1 = partials(print, 'step 1')
        self.printer2 = partials(print, 'step 2')

        self.chain(
            (None, 'distributor', queues.ThreadQueue()),
            ('distributor', 'compute', queues.ProcessQueue(5)),
            ('compute', 'compute2', queues.ProcessQueue(5)),
            ('compute', 'printer1', queues.TransferQueue()),
            ('compute2', 'printer2', queues.TransferQueue()),
            ('compute', 'compute3', queues.ProcessQueue()),
            ('compute3', None, out_queue),
            ('compute3', 'compute2'),
        )

flow = Flow()
flow

In [ ]:
with flow:
   for i in range(10):
      flow.put([i] * 2)